# Solution to the exercise from the [Dask notebook](0_Dask.ipynb)

## Exercise: Run plane segmentation in parallel using a ``Cluster`` 

Note that this section will **not** work in Google Colab.

Create a pseudo Z-stack of dimension (10, 100, 100) (ZYX)

In [16]:
planes = da.random.random((10, 100, 100), chunks=(2, 10, 10))
print(planes.shape)

(10, 100, 100)


Create a function ``analyze`` taking the z-section as a parameter.
Return the label_image and the z-section so we can identify when we parse the results.

In [ ]:
def analyze(z):
    plane = planes[z, :, :]
    smoothed_image = dask_image.ndfilters.gaussian_filter(plane, sigma=[1, 1])
    threshold_value = 0.33 * da.max(smoothed_image).compute()
    threshold_image = smoothed_image > threshold_value
    label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
    name = "z:%s" % (z)
    return label_image, name

Prepare the call

In [ ]:
def prepare_call(client, dim_z):
    futures = []
    for z in range(dim_z):
        futures.append(client.submit(analyze, z))
    return futures

Create a default local cluster

In [17]:
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster()

with Client(cluster) as client:
    # perform code
    futures = prepare_call(clien, planes.shape[0])
    results = client.gather(futures)